In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [2]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

### NMF
Non-Negative Matrix Factorization:  
Find two non-negative matrices whose product approximate non-negative matrix X

In [3]:
%%time

# n_components=500 takes too long (couldn't finish over night)

nmf = NMF(n_components=250)
nmf2 = nmf.fit_transform(tfidf_matrix)
tfidf_matrix = nmf2

Wall time: 3h 58min 10s


In [4]:
tfidf_matrix.shape

(27885L, 250L)

# save

In [5]:
#np.save('C:/Users/Michael/Desktop/network_data/scotus/nlp/nmf_250/nmf_250.npy',tfidf_matrix)

# load

In [3]:
tfidf_matrix = np.load('C:/Users/Michael/Desktop/network_data/scotus/nlp/nmf_250/nmf_250.npy')
print tfidf_matrix.shape

(27885L, 250L)


# Clustering Work
focus on largest connected component on **undirected scotus**

In [4]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

## modularity on undirected scotus
"For a given division of the network's vertices into some modules, modularity reflects the concentration of edges within modules compared with random distribution of links between all nodes regardless of modules"--*Wikipedia*

In [5]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

print mod_clust.summary()

# save clusters in pandas
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 126 clusters
Wall time: 1min 49s


In [6]:
graph_clusters_mod_score = g.modularity(mod_clust.membership)

print graph_clusters_mod_score

0.465097751507


## walktrap on undirected scotus

In [7]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

# save clusters in pandas
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 49s


In [8]:
walktrap_clusters_mod_score = g.modularity(wt_clust.membership)

print walktrap_clusters_mod_score

0.504445233426


# NLP Clustering
K-Means and Gaussian Mixture Models (GMM)

## K-Means Clustering on tf-idf (NMF)

### K-means clustering with K = 10

In [13]:
%%time

# set number of clusters
num_clusters = 10

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_tfidf_clusters = km.labels_.tolist()

Wall time: 14.3 s


In [10]:
nlp_tfidf_clusters = clusters['km_10'].tolist()

In [11]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score = g.modularity(nlp_tfidf_clusters)
print nlp_tfidf_clusters_mod_score

-0.000612948647672


### K-means clustering with K = 100

In [15]:
%%time

# set number of clusters
num_clusters = 100

# run kmeans
km2 = KMeans(n_clusters=num_clusters)
km2.fit(tfidf_matrix)

nlp_tfidf_clusters2 = km2.labels_.tolist()

Wall time: 1min 17s


In [12]:
nlp_tfidf_clusters2 = clusters['km_100'].tolist()

In [13]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score2 = g.modularity(nlp_tfidf_clusters2)
print nlp_tfidf_clusters_mod_score2

9.16074653979e-05


### K-means clustering with K = 1000

In [17]:
%%time

# set number of clusters
num_clusters = 1000

# run kmeans
km3 = KMeans(n_clusters=num_clusters)
km3.fit(tfidf_matrix)

nlp_tfidf_clusters3 = km3.labels_.tolist()

Wall time: 8min 31s


In [14]:
nlp_tfidf_clusters3 = clusters['km_1000'].tolist()

In [15]:
# modularity score of cluster
nlp_tfidf_clusters_mod_score3 = g.modularity(nlp_tfidf_clusters3)
print nlp_tfidf_clusters_mod_score3

6.13663927624e-05


## GMM Clustering on tf-idf (NMF)

### GMM Clustering with K = 10

In [19]:
%%time

# set number of components
num_components = 10

# run GMM
gmm = GaussianMixture(n_components=num_components)
gmm.fit(tfidf_matrix)

gmm_clusters = gmm.predict(tfidf_matrix).tolist()
gmm_clusters = map(int, gmm_clusters)

Wall time: 7min 28s


In [16]:
gmm_clusters = clusters['gmm_10'].tolist()

In [17]:
# modularity score of cluster
gmm_clusters_mod_score = g.modularity(gmm_clusters)
print gmm_clusters_mod_score

4.1121685105e-05


### GMM Clustering with K = 100 

In [21]:
%%time

# set number of components
num_components = 100

# run GMM
gmm2 = GaussianMixture(n_components=num_components)
gmm2.fit(tfidf_matrix)

gmm_clusters2 = gmm2.predict(tfidf_matrix).tolist()
gmm_clusters2 = map(int, gmm_clusters2)

Wall time: 32min 35s


In [18]:
gmm_clusters2 = clusters['gmm_100'].tolist()

In [19]:
# modularity score of cluster
gmm_clusters_mod_score2 = g.modularity(gmm_clusters2)
print gmm_clusters_mod_score2

-0.000119313251301


### GMM Clustering with K = 1000

In [34]:
%%time

# set number of components
num_components = 1000

# run GMM
gmm3 = GaussianMixture(n_components=num_components)
gmm3.fit(tfidf_matrix)

gmm_clusters3 = gmm3.predict(tfidf_matrix).tolist()
gmm_clusters3 = map(int, gmm_clusters3)

Wall time: 3h 52min 18s


In [20]:
gmm_clusters3 = clusters['gmm_1000'].tolist()

In [21]:
# modularity score of cluster
gmm_clusters_mod_score3 = g.modularity(gmm_clusters3)
print gmm_clusters_mod_score3

-0.000174761939599


## Hierarchical Clustering on tf-idf (NMF)

### Hierarchical Clustering with K = 10

In [36]:
%%time

# set number of components
num_clusters = 10

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters = hc.fit_predict(tfidf_matrix).tolist()
hc_clusters = map(int, hc_clusters)

Wall time: 16min 3s


In [22]:
hc_clusters = clusters['hc_10'].tolist()

In [23]:
# modularity score of cluster
hc_clusters_mod_score = g.modularity(hc_clusters)
print hc_clusters_mod_score

-0.000189961152751


### Hierarchical Clustering with K = 100

In [38]:
%%time

# set number of components
num_clusters = 100

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc2 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters2 = hc2.fit_predict(tfidf_matrix).tolist()
hc_clusters2 = map(int, hc_clusters2)

Wall time: 10min 16s


In [24]:
hc_clusters2 = clusters['hc_100'].tolist()

In [25]:
# modularity score of cluster
hc_clusters_mod_score2 = g.modularity(hc_clusters2)
print hc_clusters_mod_score2

0.000312383817542


### Hierarchical Clustering with K = 1000

In [ ]:
%%time

# set number of components
num_clusters = 1000

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc3 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters3 = hc3.fit_predict(tfidf_matrix).tolist()
hc_clusters3 = map(int, hc_clusters3)

In [26]:
hc_clusters3 = clusters['hc_1000'].tolist()

In [27]:
# modularity score of cluster
hc_clusters_mod_score3 = g.modularity(hc_clusters3)
print hc_clusters_mod_score3

-0.000177753188175


## Compare NLP clustering (tfidf) vs graph clustering

In [36]:
#clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])
clusters = pd.DataFrame(index=op_id_to_bow_id, columns=[])

# add in communities 
clusters['mod'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['mod'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['mod'] = clusters['mod'].astype(np.int)

# add in walktrap clusters
clusters['wt'] = walktrap_clusters
clusters['wt'].fillna(max(walktrap_clusters) + 1, inplace=True)
clusters['wt'] = clusters['wt'].astype(np.int)

# add in NLP clusters
clusters['km_10'] = nlp_tfidf_clusters
clusters['km_100'] = nlp_tfidf_clusters2
clusters['km_1000'] = nlp_tfidf_clusters3

clusters['gmm_10'] = gmm_clusters
clusters['gmm_100'] = gmm_clusters2
clusters['gmm_1000'] = gmm_clusters3

clusters['hc_10'] = hc_clusters
clusters['hc_100'] = hc_clusters2
clusters['hc_1000'] = hc_clusters3

#clusters.to_csv("clusters_NMF.csv")
clusters.to_csv(csv_dir + "clusters_NMF.csv")

### load cluster csv saved in current directory

In [34]:
clusters = pd.read_csv(csv_dir + 'clusters_NMF.csv')
clusters

,Unnamed: 0,mod,wt,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
0,145658,1,5,5,71,356,0,37,807,1,86,74
1,89370,3,294,4,71,557,0,37,145,1,61,267
2,89371,0,35,4,73,546,7,11,560,1,85,144
3,89372,0,3,4,12,719,0,13,523,1,1,814
4,89373,0,3,9,10,602,6,74,681,1,47,79
5,89374,2,4,4,10,198,9,9,270,1,47,658
6,89375,2,5,4,16,576,3,74,137,1,2,811
7,89376,2,6,4,12,673,6,40,498,3,29,221
8,89377,2,7,4,56,714,5,55,598,1,1,875
9,89378,2,7,5,85,966,6,40,925,1,22,114


# MAJOR CORRECTION (modularity and walktrap have different indices than NLP clusters!!!)
make separate csv for mod and wt column  
delete these two columns from original csv

In [35]:
clusters = clusters.set_index('Unnamed: 0')
clusters.index.name=None
clusters = clusters[["km_10","km_100","km_1000","gmm_10","gmm_100","gmm_1000","hc_10","hc_100","hc_1000"]]
clusters.to_csv(csv_dir + 'clusters_NMF.csv')

In [36]:
clusters

,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
145658,5,71,356,0,37,807,1,86,74
89370,4,71,557,0,37,145,1,61,267
89371,4,73,546,7,11,560,1,85,144
89372,4,12,719,0,13,523,1,1,814
89373,9,10,602,6,74,681,1,47,79
89374,4,10,198,9,9,270,1,47,658
89375,4,16,576,3,74,137,1,2,811
89376,4,12,673,6,40,498,3,29,221
89377,4,56,714,5,55,598,1,1,875
89378,5,85,966,6,40,925,1,22,114


# Modularity Scores of Clusters

In [30]:
clusters_strings = ['mod', 'wt', 'km_10', 'km_100', 'km_1000', 'gmm_10', 'gmm_100', 'gmm_1000', 'hc_10', 'hc_100', 'hc_1000']
mod_scores = [graph_clusters_mod_score, walktrap_clusters_mod_score, nlp_tfidf_clusters_mod_score, nlp_tfidf_clusters_mod_score2,
             nlp_tfidf_clusters_mod_score3, gmm_clusters_mod_score, gmm_clusters_mod_score2, gmm_clusters_mod_score3,
             hc_clusters_mod_score, hc_clusters_mod_score2, hc_clusters_mod_score3]

clusters = pd.DataFrame(index=clusters_strings)
clusters['modularity_score'] = mod_scores

clusters.to_csv(csv_dir + "clusters_NMF_mod_score.csv")

In [31]:
clusters_mod_score = pd.read_csv(csv_dir + 'clusters_NMF_mod_score.csv')
clusters_mod_score

,Unnamed: 0,modularity_score
0,mod,0.465098
1,wt,0.504445
2,km_10,-0.000613
3,km_100,0.000092
4,km_1000,0.000061
5,gmm_10,0.000041
6,gmm_100,-0.000119
7,gmm_1000,-0.000175
8,hc_10,-0.000190
9,hc_100,0.000312


# normalized_mutual_information_score (nmi) of
## 1. modularity vs. walktrap
## 2. every combination of nlp_clusters
note: can't do nmi(graph_cluster, nlp_cluster) because len(graph_cluster) != len(nlp_cluster)  
http://scikit-learn.org/stable/modules/classes.html#clustering-metrics

In [32]:
string_list = ["km_10", "km_100", "km_1000", "gmm_10", "gmm_100", "gmm_1000", 
               "hc_10", "hc_100", "hc_1000"]
clusters_list = [nlp_tfidf_clusters, nlp_tfidf_clusters2, nlp_tfidf_clusters3,
                gmm_clusters, gmm_clusters2, gmm_clusters3, hc_clusters, hc_clusters2, hc_clusters3]

nums = range(0,9)
combs = list(combinations(nums,2))

'''
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm
'''

combinations_list = []
nmi_scores = []
ami_scores = []
mi_scores = []
ar_scores = []
#ch_scores = []
cs_scores = []
fm_scores = []
#hcvm_scores = []
hs_scores = []
#ss_scores = []
#ss2_scores = []
vm_scores = []

combinations_list.append("mod vs. wt")
nmi_scores.append(nmi(graph_clusters, walktrap_clusters))
ami_scores.append(ami(graph_clusters, walktrap_clusters))
mi_scores.append(mi(graph_clusters, walktrap_clusters))
ar_scores.append(ar(graph_clusters, walktrap_clusters))
#ch_scores.append(ch(graph_clusters, walktrap_clusters))
cs_scores.append(cs(graph_clusters, walktrap_clusters))
fm_scores.append(fm(graph_clusters, walktrap_clusters))
#hcvm_scores.append(hcvm(graph_clusters, walktrap_clusters))
hs_scores.append(hs(graph_clusters, walktrap_clusters))
#ss_scores.append(ss(graph_clusters, walktrap_clusters))
#ss2_scores.append(ss2(graph_clusters, walktrap_clusters))
vm_scores.append(vm(graph_clusters, walktrap_clusters))

for i in combs:
    combinations_list.append(string_list[i[0]] + " vs. " + string_list[i[1]])
    nmi_scores.append(nmi(clusters_list[i[0]], clusters_list[i[1]]))
    ami_scores.append(ami(clusters_list[i[0]], clusters_list[i[1]]))
    mi_scores.append(mi(clusters_list[i[0]], clusters_list[i[1]]))
    ar_scores.append(ar(clusters_list[i[0]], clusters_list[i[1]]))
    #ch_scores.append(ch(clusters_list[i[0]], clusters_list[i[1]]))
    cs_scores.append(cs(clusters_list[i[0]], clusters_list[i[1]]))
    fm_scores.append(fm(clusters_list[i[0]], clusters_list[i[1]]))
    #hcvm_scores.append(hcvm(clusters_list[i[0]], clusters_list[i[1]]))
    hs_scores.append(hs(clusters_list[i[0]], clusters_list[i[1]]))
    #ss_scores.append(ss(clusters_list[i[0]], clusters_list[i[1]]))
    #ss2_scores.append(ss2(clusters_list[i[0]], clusters_list[i[1]]))
    vm_scores.append(vm(clusters_list[i[0]], clusters_list[i[1]]))
    
clusters = pd.DataFrame(index=combinations_list)
clusters["nmi_score"] = nmi_scores
clusters["ami_score"] = ami_scores
clusters["mi_score"] = mi_scores
clusters["ar_score"] = ar_scores
clusters["cs_score"] = cs_scores
clusters["fm_score"] = fm_scores
clusters["hs_score"] = hs_scores
clusters["vm_score"] = vm_scores
clusters.to_csv(csv_dir + "clusters_NMF_nmi_score.csv")

In [33]:
clusters_nmi_score = pd.read_csv(csv_dir + 'clusters_NMF_nmi_score.csv')
clusters_nmi_score

,Unnamed: 0,nmi_score,ami_score,mi_score,ar_score,cs_score,fm_score,hs_score,vm_score
0,mod vs. wt,0.384960,0.184582,1.001603,0.088432,0.222527,0.233443,0.665962,0.333588
1,km_10 vs. km_100,0.350264,0.171115,0.688405,0.052523,0.174561,0.248108,0.702818,0.279662
2,km_10 vs. km_1000,0.328390,0.113586,0.817500,0.004351,0.129209,0.071277,0.834616,0.223775
3,km_10 vs. gmm_10,0.121036,0.079409,0.179337,0.021406,0.080012,0.275075,0.183092,0.111360
4,km_10 vs. gmm_100,0.214775,0.098151,0.444686,0.008749,0.101605,0.119330,0.453996,0.166048
5,km_10 vs. gmm_1000,0.321603,0.110128,0.805417,0.002933,0.125783,0.059121,0.822280,0.218189
6,km_10 vs. hc_10,0.031379,0.021563,0.022591,0.019849,0.042692,0.677536,0.023064,0.029948
7,km_10 vs. hc_100,0.115353,0.053770,0.225694,0.013255,0.057749,0.189165,0.230419,0.092352
8,km_10 vs. hc_1000,0.161463,0.045515,0.406987,0.001051,0.062743,0.043245,0.415508,0.109023
9,km_100 vs. km_1000,0.659490,0.442556,3.294228,0.101925,0.520666,0.206207,0.835328,0.641488
